In [1]:
import pandas as pd
import numpy as np
import math
from datetime import datetime

In [2]:
df = pd.read_csv('../Data/Absenteeism_at_work.csv', sep=';')
df.columns = [x.strip().replace(' ', '_').lower() for x in df.columns]

df.head()

,id,reason_for_absence,month_of_absence,day_of_the_week,seasons,transportation_expense,distance_from_residence_to_work,service_time,age,work_load_average/day,...,disciplinary_failure,education,son,social_drinker,social_smoker,pet,weight,height,body_mass_index,absenteeism_time_in_hours
0,11,26,7,3,1,289,36,13,33,239.554,...,0,1,2,1,0,1,90,172,30,4
1,36,0,7,3,1,118,13,18,50,239.554,...,1,1,1,1,0,0,98,178,31,0
2,3,23,7,4,1,179,51,18,38,239.554,...,0,1,0,1,0,0,89,170,31,2
3,7,7,7,5,1,279,5,14,39,239.554,...,0,1,2,1,1,0,68,168,24,4
4,11,23,7,5,1,289,36,13,33,239.554,...,0,1,2,1,0,1,90,172,30,2


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740 entries, 0 to 739
Data columns (total 21 columns):
id                                 740 non-null int64
reason_for_absence                 740 non-null int64
month_of_absence                   740 non-null int64
day_of_the_week                    740 non-null int64
seasons                            740 non-null int64
transportation_expense             740 non-null int64
distance_from_residence_to_work    740 non-null int64
service_time                       740 non-null int64
age                                740 non-null int64
work_load_average/day              740 non-null float64
hit_target                         740 non-null int64
disciplinary_failure               740 non-null int64
education                          740 non-null int64
son                                740 non-null int64
social_drinker                     740 non-null int64
social_smoker                      740 non-null int64
pet                          

In [91]:
df['seasons'] = df['seasons'].astype('category')
df['reason_for_absence'] = df['reason_for_absence'].astype('category')

In [78]:
df[df['seasons']==3].month_of_absence.unique()

df['seasons'] == 1
for i in range(1,5):
    df['seasons_'+str(i)] = df['seasons']==i
    df['seasons_'+str(i)] = df['seasons_'+str(i)].astype(int)
    
df['seasons'].unique()

df[df['reason_for_absence']==0].head()

,id,reason_for_absence,month_of_absence,day_of_the_week,seasons,transportation_expense,distance_from_residence_to_work,service_time,age,work_load_average/day,...,social_smoker,pet,weight,height,body_mass_index,absenteeism_time_in_hours,seasons_1,seasons_2,seasons_3,seasons_4
1,36,0,7,3,1,118,13,18,50,239.554,...,0,0,98,178,31,0,1,0,0,0
50,20,0,9,2,4,260,50,11,36,241.476,...,0,0,65,168,23,0,0,0,0,1
51,29,0,9,2,4,225,26,9,28,241.476,...,0,2,69,169,24,0,0,0,0,1
54,11,0,9,3,4,289,36,13,33,241.476,...,0,1,90,172,30,0,0,0,0,1
55,36,0,9,3,4,118,13,18,50,241.476,...,0,0,98,178,31,0,0,0,0,1


In [86]:
absence_reason_dict = {
    '0' : ['unknown_0', 0],
    '1' : ['infectious_parasitic_1', 1],
    '2' : ['neoplasms_2', 1],
    '3' : ['blood_immune_3', 1],
    '4' : ['endo_nutritional_metabolic_4', 1],
    '5' : ['mental_behavioral_5', 1],
    '6' : ['nervous_6', 1],
    '7' : ['eye_adnexa_7', 1],
    '8' : ['ear_mastoid_8', 1],
    '9' : ['circulatory_9', 1],
    '10' : ['respiratory_10', 1],
    '11' : ['digestive_11', 1],
    '12' : ['skin_subcutaneous_12', 1],
    '13' : ['musculoskeletal_connective_13', 1],
    '14' : ['genitourinary_14', 1],
    '15' : ['puerperium_pregnancy_15', 1],
    '16' : ['perinatal_16', 1],
    '17' : ['Congenital_17', 1],
    '18' : ['abnormal_unclassified_18', 1],
    '19' : ['injury_poisoning_19', 1],
    '20' : ['morbidity_mortality_20', 1],
    '21' : ['health_status_factors_21', 1],
    '22' : ['patient_follow_up_22', 2],
    '23' : ['medical_consultation_23', 2],
    '24' : ['blood_donation_24', 2],
    '25' : ['lab_exam_25', 2],
    '26' : ['unjustifiied_26', 2],
    '27' : ['physiotherapy_27', 2],
    '28' : ['dental_condultation_28', 2]
}



In [88]:
df['reason_for_absence_description'] = df['reason_for_absence'].map(lambda x : (absence_reason_dict.get(str(x))[0]))
df['reason_for_absence_type_unknown'] = df['reason_for_absence'].map(lambda x : int(absence_reason_dict.get(str(x))[1]==0))
df['reason_for_absence_type_icd'] = df['reason_for_absence'].map(lambda x : int(absence_reason_dict.get(str(x))[1]==1))
df['reason_for_absence_type_nonicd'] = df['reason_for_absence'].map(lambda x : int(absence_reason_dict.get(str(x))[1]==2))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740 entries, 0 to 739
Data columns (total 25 columns):
id                                 740 non-null int64
reason_for_absence                 740 non-null category
month_of_absence                   740 non-null int64
day_of_the_week                    740 non-null int64
seasons                            740 non-null category
transportation_expense             740 non-null int64
distance_from_residence_to_work    740 non-null int64
service_time                       740 non-null int64
age                                740 non-null int64
work_load_average/day              740 non-null float64
hit_target                         740 non-null int64
disciplinary_failure               740 non-null int64
education                          740 non-null int64
son                                740 non-null int64
social_drinker                     740 non-null int64
social_smoker                      740 non-null int64
pet                    

In [95]:
df['month_of_absence'] = df['month_of_absence'] + 1

In [102]:
df.id.unique().shape

(36,)